In [ ]:
#구글코랩 mount

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#모델에 필요한 패키지를 설치합니다.

!pip install transformers==4.10.0
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 14.5 MB/s 
     |████████████████████████████████| 163 kB 71.9 MB/s 
     |████████████████████████████████| 3.3 MB 58.9 MB/s 
     |████████████████████████████████| 880 kB 78.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=3a86ed4d8457f90f6101f99bc9a932235552554c05e715b17ae79511ccebbb1f
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 708 kB 14.9 MB/s 
     |████████████████████████████████| 5.9 MB 66.3 MB/s 
     |████████████████████████████████| 529 kB 96.1 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstal

In [ ]:
#관련 모듈을 호출합니다.

import os
os.chdir('/content/drive/MyDrive/Hana_project')
import pandas as pd
from sklearn.model_selection import train_test_split
import collections
from tqdm import tqdm
import re

from arguments import TrainArguments
from utill import set_seed
from utill import data_collator

from task import QATask

from corpus import QACorpus, QADataset
from corpus import QAFeatures

from transformers import AutoTokenizer, AutoConfig, AutoModelForQuestionAnswering

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
#misc
# {park2020koelectra,
#   author = {Park, Jangwon},
#   title = {KoELECTRA: Pretrained ELECTRA Model for Korean},
#   year = {2020},
#   publisher = {GitHub},
#   journal = {GitHub repository},
#   howpublished = {\url{https://github.com/monologg/KoELECTRA}}


In [ ]:
import pandas as pd

In [ ]:
#학습 데이터를 생성합니다.

raw_data = pd.read_excel('/content/drive/MyDrive/Hana_project/data/hana_power_v1/Hana_power_v5_raw.xlsx')

raw_data = raw_data.iloc[:,1:]

raw_data = raw_data.dropna()

raw_data['start_position_character'] = raw_data['start_position_character'].astype('int')

X_train, X_test = train_test_split(raw_data, test_size = 0.3, random_state=2022)

X_train.to_csv('/content/drive/MyDrive/Hana_project/data/hana_power_v1/Hana_power_v1_train.csv', encoding = 'cp949')

X_test.to_csv('/content/drive/MyDrive/Hana_project/data/hana_power_v1/Hana_power_v1_dev.csv', encoding = 'cp949')

In [ ]:
raw_data

,content,fund,class,question_text,answer_text,start_position_character
0,"수익권의 분할) ①이 투자신탁의 수익권은 1좌의 단위로 균등하게 분할하며, 수익증권...",HDC더블리자드지수연계증권투자신탁NHE-23호[ELS-파생형],수익권의 분할,수익권의 분할이란?,"이 투자신탁의 수익권은 1좌의 단위로 균등하게 분할하며, 수익증권으로 표시한다.",11
1,"수익권의 분할) ①이 투자신탁의 수익권은 1좌의 단위로 균등하게 분할하며, 수익증권...",HDC더블리자드지수연계증권투자신탁NHE-23호[ELS-파생형],수익권의 분할,수익권의 분할이란?,"이 투자신탁의 수익권은 1좌의 단위로 균등하게 분할하며, 수익증권으로 표시한다.",11
2,수익자명부 및 실질수익자명부) ①집합투자업자는 수익자명부 작성에 관한 업무를 한국예...,HDC더블리자드지수연계증권투자신탁NHE-23호[ELS-파생형],수익자명부 및 실질수익자명부,수익자명부의 위탁기업은?,집합투자업자는 수익자명부 작성에 관한 업무를 한국예탁결제원에 위탁하여야 한다.,19
3,수익자명부 및 실질수익자명부) ①집합투자업자는 수익자명부 작성에 관한 업무를 한국예...,HDC더블리자드지수연계증권투자신탁NHE-23호[ELS-파생형],수익자명부 및 실질수익자명부,수익자명부의 위탁기업은?,집합투자업자는 수익자명부 작성에 관한 업무를 한국예탁결제원에 위탁하여야 한다.,19
4,수익증권의 발행 및 예탁) ①집합투자업자는 제6조 및 제7조의 규정에 의한 투자신탁...,HDC더블리자드지수연계증권투자신탁NHE-23호[ELS-파생형],수익증권의 발행 및 예탁,수익증권의 발행이란?,집합투자업자는 제6조 및 제7조의 규정에 의한 투자신탁의 최초설정 및 추가설정에 의...,17
...,...,...,...,...,...,...
4526,의결권 등) ①의결권은 수익증권 1좌마다 1개로 한다 ②수익자가 대리인으로 하여금...,삼성애버딘미국중소형증권자투자신탁(UH)[주식-재간접],의결권 등,의결은 어느정도 주는거야,수익증권 1좌마다 1개,14
4527,"용어의 정의) 이 신탁계약에서 사용하는 용어의 정의는 다음 각 호와 같다. 다만, ...",하나UBS글로벌4차산업1등주자산배분증권자투자신탁[주혼],용어의 정의,수익자가 뭐야,이 투자신탁의 수익증권을 보유하는 자,107
4528,"손해배상책임) ①집합투자업자, 신탁업자는 법령?신탁계약?투자설명서에 위반하는 행위를...",하나UBS글로벌4차산업1등주자산배분증권자투자신탁[주혼],손해배상책임,손해배상 해줘?,투자설명서에 위반하는 행위를 하거나 그 업무를 소홀히 하여 수익자에게 손해를 발생시...,32
4529,손익의 귀속 등) 투자신탁재산의 운용과 관련하여 집합투자업자의 지시에 따라 발생한 ...,삼성애버딘미국중소형증권자투자신탁(UH)[주식-재간접],손익의 귀속 등,이익은 누구한테 가,집합투자업자의 지시에 따라 발생한 이익 및 손실은 모두 이 투자신탁에 계상되고 수익...,28


In [ ]:
#Arguments를 설정해줍니다.

args = TrainArguments(
    pretrained_model_name="monologg/koelectra-base-v3-discriminator",
    downstream_corpus_name="hana_power_v1",
    downstream_model_dir="/content/drive/MyDrive/Hana_project/model/",
    max_seq_length=128,
    max_query_length=32,
    doc_stride=32,
    batch_size=32,
    learning_rate=5e-5,
    epochs=10,
    seed=7,
)

#seet 설정
set_seed(args)

set seed: 7


In [ ]:
#Pretrained 모델에서 Tokenizer를 가져옵니다.

tokenizer = AutoTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
    use_fast=False
)

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

In [ ]:
# 스페셜 토큰을 코트나이저에 넣습니다.
user_defined_symbols = ['[BOS]','[EOS]','[UNK0]','[UNK1]','[UNK2]','[UNK3]','[UNK4]','[UNK5]','[UNK6]','[UNK7]','[UNK8]','[UNK9]']
unused_token_num = 200
unused_list = ['[unused{}]'.format(n) for n in range(unused_token_num)]
user_defined_symbols = user_defined_symbols + unused_list

# tokenizer에 special token 추가
special_tokens_dict = {'additional_special_tokens': user_defined_symbols}
tokenizer.add_special_tokens(special_tokens_dict)

# check tokenizer vocab with special tokens
print('check special tokens : %s'%tokenizer.all_special_tokens[:20])

check special tokens : ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[BOS]', '[EOS]', '[UNK0]', '[UNK1]', '[UNK2]', '[UNK3]', '[UNK4]', '[UNK5]', '[UNK6]', '[UNK7]', '[UNK8]', '[UNK9]', '[unused0]', '[unused1]', '[unused2]']


In [ ]:
#학습 데이터 셋에서 Corpus를 추출하고 Train_dataloader에 데이터를 넣습니다.
corpus = QACorpus()

train_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

convert squad examples to features: 100%|██████████| 3170/3170 [03:03<00:00, 17.25it/s]


In [ ]:
#test 데이터 셋에서 Corpus를 추출하고 Train_dataloader에 데이터를 넣습니다.

val_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="val",
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

convert squad examples to features: 100%|██████████| 1359/1359 [01:20<00:00, 16.89it/s]


In [ ]:
#모델 설정

pretrained_model_config = AutoConfig.from_pretrained(args.pretrained_model_name,)

model = AutoModelForQuestionAnswering.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)



Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['qa_outputs.weight'

In [ ]:
#QnA 학습에 쓰일 Taks를 정의합니다.

task = QATask(model, args)

In [ ]:
import torch

def get_trainer(args, return_trainer_only=True):

    ckpt_path = os.path.abspath(args.downstream_model_checkpoint_fpath)
    os.makedirs(ckpt_path, exist_ok=True)
    checkpoint_callback = ModelCheckpoint(
        dirpath=ckpt_path,
        save_top_k=args.save_top_k,
        monitor=args.monitor.split()[1],
        mode=args.monitor.split()[0],
        filename='{epoch}-{val_loss:.2f}',
    )
    
    trainer = Trainer(
        max_epochs=args.epochs,
        fast_dev_run=args.test_mode,
        num_sanity_val_steps=None if args.test_mode else 0,
        callbacks=[checkpoint_callback],
        default_root_dir=ckpt_path,

        # For GPU Setup
        deterministic=torch.cuda.is_available() and args.seed is not None,
        gpus=torch.cuda.device_count() if torch.cuda.is_available() else None,
        precision=16 if args.fp16 else 32,
        
        )
    if return_trainer_only:
        return trainer

    else:
        return checkpoint_callback, trainer

In [ ]:
#모델 학습을 진행합니다.

trainer = get_trainer(args)

trainer.fit(task, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /content/drive/MyDrive/Hana_project/model/checkpoint exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LO

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
#학습된 모델을 저장합니다.

torch.save(model, f'model/model_QA7_koelectra.pt')

In [ ]:
#학습된 모델을 호출합니다.

model = torch.load(f'/content/drive/MyDrive/Hana_project/model/model_QA4_koelectra-base-v3.pt')

In [ ]:
#평가모드로 전환합니다.

model.eval()

ElectraForQuestionAnswering(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768

In [ ]:
def predict(contexts, questions):
  result = []

  for context, question in tqdm(zip(contexts, questions)):
    truncated_query = tokenizer.encode(
        question,
        add_special_tokens=False,
        truncation=True,
        max_length=args.max_query_length
    )
    inputs = tokenizer.encode_plus(
        text=truncated_query,
        text_pair=context,
        truncation="only_second",
        padding="max_length",
        max_length=args.max_seq_length,
        return_token_type_ids=True,
    )

    with torch.no_grad():
      
        outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})

        start_pred = outputs.start_logits.argmax(dim=-1).item()

        end_pred = outputs.end_logits.argmax(dim=-1).item()

        pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1]).replace('[CLS]','').replace('[SEP]','.').replace('[UNK]','')

        result.append(pred_text)
        
      
  return result

In [ ]:
#모델 평가와 관련된 함수를 정의합니다.


def normalize_answer(sentence):
  ret = re.sub('[-=+,#/\:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…\①]', '', sentence)
  ret = re.sub(r'[\n\r\t]+', ' ', sentence)
  ret = ret.strip()
  return ret

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
  return int(a_gold == a_pred)

def compute_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

In [ ]:
#평가 데이터를 호출합니다.

x_valid = pd.read_csv('/content/drive/MyDrive/Hana_project/data/hana_power_v1/Hana_power_v1_dev.csv',encoding = 'cp949')

ctx = x_valid['content'].tolist()
qus = x_valid['question_text'].tolist()
ans = x_valid['answer_text'].tolist()

In [ ]:
# 평가 데이터에 대해 예측합니다.

prds =  predict(ctx, qus)

1359it [05:36,  4.04it/s]


In [ ]:
prds = pd.DataFrame(prds)[0].apply(lambda x : x.replace('①',''))

In [ ]:
pd.DataFrame(prds).to_csv('/content/drive/MyDrive/Hana_project/data/hana_power_v1/prediction_v4.csv')

In [ ]:
_len, f1scores, emscores = len(prds), [], []
for i in tqdm(range(_len)):
  f1scores.append(compute_f1(ans[i], prds[i]))
  emscores.append(compute_exact(ans[i], prds[i]))

100%|██████████| 1359/1359 [00:00<00:00, 31706.10it/s]


In [ ]:
#모델 평가 점수를 보여줍니다.

scores = [0, 0]
for i in range(_len):
  scores[0] += f1scores[i]
  scores[1] += emscores[i]

print(f'[Calculated Scores for {_len} data]')
print(f'===> F1: {scores[0]/_len}, EM: {scores[1]/_len}')

[Calculated Scores for 1359 data]
===> F1: 0.7693836141243804, EM: 0.35688005886681384


In [ ]:
# 모델을 저장합니다.

torch.save(model, f'model/model_QA.pt')